In [19]:
import $ivy.`com.amazonaws:aws-java-sdk:1.11.22`

import $ivy.$                                   

In [20]:
val region = "us-east-1"

region: String = "us-east-1"

In [21]:
import com.amazonaws.services.ec2._
import com.amazonaws.regions._
val ec2 = new AmazonEC2Client()
val regions = Regions.fromName(region)
ec2.setRegion(Region.getRegion(regions))

import com.amazonaws.services.ec2._

import com.amazonaws.regions._

ec2: AmazonEC2Client = com.amazonaws.services.ec2.AmazonEC2Client@55c1ae8f
regions: Regions = US_EAST_1

In [22]:
import collection.JavaConverters._

val instances = for {
 reservation <- ec2.describeInstances.getReservations.asScala
 instance <- reservation.getInstances.asScala
} yield {
    instance
}
val ids = instances.filter(_.getState.getName == "running").map(_.getInstanceId)
val stopped = instances.filter(_.getState.getName == "stopped").map(_.getInstanceId)
val ips = instances.filter(_.getState.getName == "running").map(i => i.getPrivateIpAddress -> i.getPublicIpAddress)

import collection.JavaConverters._


instances: collection.mutable.Buffer[model.Instance] = ArrayBuffer(
  {InstanceId: i-0b61965d913df17ad,ImageId: ami-10f53a06,State: {Code: 48,Name: terminated},PrivateDnsName: ,PublicDnsName: ,StateTransitionReason: User initiated (2017-04-18 13:57:07 GMT),KeyName: atomic,AmiLaunchIndex: 0,ProductCodes: [],InstanceType: t2.small,LaunchTime: Sat Apr 15 09:22:55 CEST 2017,Placement: {AvailabilityZone: us-east-1d,GroupName: ,Tenancy: default,},Monitoring: {State: disabled},StateReason: {Code: Client.UserInitiatedShutdown,Message: Client.UserInitiatedShutdown: User initiated shutdown},Architecture: x86_64,RootDeviceType: ebs,RootDeviceName: /dev/sda1,BlockDeviceMappings: [],VirtualizationType: hvm,ClientToken: test-Master-DSDT14T074QI,Tags: [{Key: Application,Value: arn:aws:cloudformation:us-east-1:794403675119:stack/test/984818d0-210e-11e7-932e-50d5cd0dfcc6}, {Key: aws:cloudformation:stack-name,Value: test}, {Key: aws:cloudformation:logi...
ids: collec

# Building the inventory

In [23]:
val inventory = Seq(
            "[all:vars]",
            s"kube_master_host=${ips.head._2}",
            s"kube_master=${ips.head._1}",    
            "[aws]"
        ) ++ ips.map(x => s"${x._2} local_ip=${x._1}") ++
          Seq("[masters]", s"${ips.head._2} local_ip=${ips.head._1}") ++
          Seq("[nodes]") ++
          ips.tail.map(t => s"${t._2} local_ip=${t._1}")


inventory: Seq[String] = List(
  "[all:vars]",
  "kube_master_host=50.16.73.7",
  "kube_master=10.0.0.73",
  "[aws]",
  "50.16.73.7 local_ip=10.0.0.73",
  "52.207.57.11 local_ip=10.0.0.199",
  "54.236.43.68 local_ip=10.0.0.102",
  "[masters]",
  "50.16.73.7 local_ip=10.0.0.73",
  "[nodes]",
  "52.207.57.11 local_ip=10.0.0.199",
...

In [24]:
import ammonite.ops._
val inv = pwd/'aws/"inventory"
rm! inv
write(inv, inventory.mkString("\n"))
show(inventory)

List(
  "[all:vars]",
  "kube_master_host=50.16.73.7",
  "kube_master=10.0.0.73",
  "[aws]",
  "50.16.73.7 local_ip=10.0.0.73",
  "52.207.57.11 local_ip=10.0.0.199",
  "54.236.43.68 local_ip=10.0.0.102",
  "[masters]",
  "50.16.73.7 local_ip=10.0.0.73",
  "[nodes]",
  "52.207.57.11 local_ip=10.0.0.199",
  "54.236.43.68 local_ip=10.0.0.102"
)


import ammonite.ops._

inv: Path = /Users/msciab/Dropbox/Work/MacBookPro/MicheleSciabarra/Mosaico/cloud/aws/inventory

# Kubectl config

In [25]:
 val kubecfg = s"""
apiVersion: v1
clusters:
- cluster:
    server: http://${ips.head._2}:8080
  name: aws
contexts:
- context:
    cluster: aws
    user: ""
  name: aws
current-context: aws
"""

kubecfg: String = """

apiVersion: v1
clusters:
- cluster:
    server: http://50.16.73.7:8080
  name: aws
contexts:
- context:
    cluster: aws
    user: ""
  name: aws
...

In [26]:
val cfg = pwd/'aws/"kubeconfig"
rm! cfg
write(cfg, kubecfg)

cfg: Path = /Users/msciab/Dropbox/Work/MacBookPro/MicheleSciabarra/Mosaico/cloud/aws/kubeconfig